In [ ]:
%load_ext autoreload
%autoreload 2
#class variables are initialized in config
from config import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import logging
import os
from emailscraper_app.modules.Sending_Emails.sends import EmailConfig
from emailscraper_app.modules.Sending_Emails.sends import SendMail
from emailscraper_app.modules.Sending_Emails.html_email_strings.schools_sport_focus import get_template
#This import dictates template


#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Sender.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)


df = pd.read_csv('phony.csv')
# ---------------------------

def blast(email_config, df, user, test=False):


    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break once end_point has surpassed.
    while len(df) >= 1:

        next_50 = SendMail.get_next_50(df)
        try:
            new_point = next_50.index[0]
            end_point = next_50.index[-1]   
            print(new_point, end_point) 
            logging.info(f'\n\nStarting email send at index {new_point}, and ending at index {end_point}')
        except IndexError:
            print('List has been fully iterated through')

         # Check if the end_point surpasses 1500
        if end_point >= 1500:
            logging.info('Reached the end_point limit of 1500. Exited the loop.')
            print('Reached the end_point limit of 1500. Exited the loop.')
            break
        
        # Process the emails
        email_history = SendMail.process(next_50, email_config, user, test)
        email_history.to_csv('output.csv', index = False)

        
        if test:
            logging.info('Breaking loop after sending to personal emails due to test')
            print('Breaking loop after sending to personal emails due to test')
            break  # Terminate after one iteration if test=True
        
        interval_seconds = 30
        # Wait for the specified interval before the next iteration
        try:
            time.sleep(interval_seconds)
            logging.info(f'Sleeping for {interval_seconds} seconds completed')
            print(f'Sleeping for {interval_seconds} seconds')
        except:
            logging.info('Issue with the sleep')
            print('Issue with the sleep')
    
    os.remove('output.csv')

#If test ARG is True it will keep sending to personal email
#& output.csv must not exist at the beginning of new runs
blast(email_config, df, test=True)


In [ ]:
%load_ext autoreload
%autoreload 2
#class variables are initialized in config
import django
from django.http import HttpRequest
import os

# Setting up Django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'emailscraper_proj.settings')
django.setup()
from emailscraper_app.modules.Sending_Emails.sends import SendMail
from config import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import logging




#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

# logging.basicConfig(filename= logpath_creation + '\\Email_Sender.log', level=logging.INFO,
#                     format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)



# ---------------------------

def blast(email_config, df,  test=True):

    print('Blast has been called')
    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break once end_point has surpassed.
    while len(df) >= 1:

        print(df.head())
        print(f'Here is the email config passed into blast func {email_config}')

        # next_50 = SendMail.get_next_50(df, email_config)

        # print(f'Here is the next_50 frame {next_50.head()}')

        # try:
        #     new_point = next_50.index[0]
        #     end_point = next_50.index[-1]   
        #     print(new_point, end_point) 
        #     logging.info(f'\n\nStarting email send at index {new_point}, and ending at index {end_point}')
        # except IndexError:
        #     print('List has been fully iterated through')

        #  # Check if the end_point surpasses 1500
        # if end_point >= 1500:
        #     logging.info('Reached the end_point limit of 1500. Exited the loop.')
        #     print('Reached the end_point limit of 1500. Exited the loop.')
        #     break
        
        # SMTP conn populates directly in process, & if test is True it will only send two emails to personal accounts
        # email_history = SendMail.process(next_50, email_config, test)
        email_history = SendMail.process(df, email_config, test) 

        if test:
            logging.info('Breaking loop after sending to personal emails due to test')
            break  # Terminate after one iteration if test=True
        else:
            logging.info('Test is false')
        
        interval_seconds = 30
        # Wait for the specified interval before the next iteration
        try:
            time.sleep(interval_seconds)
            logging.info(f'Sleeping for {interval_seconds} seconds completed')
            print(f'Sleeping for {interval_seconds} seconds')
        except:
            logging.info('Issue with the sleep')
            print('Issue with the sleep')




email_config = {'EMAIL_ADDRESS_FROM': '2015samtaylor@gmail.com', 
                'EMAIL_PASS': 'feqdwowrmaqthjkx', 
                'server': 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com',
                  'database': 'emailcampaign', 
                  'table_name': 'email_history', 
                  'email_subject_line': 'Local Supplier for Baseball Apparel', 
                  'email_campaign_name': 'Local Supplier Baseball', 
                  'contact_column': 'Email', 
                  'db_pass': 'Pretty11', 
                  'db_user': 'admin', 
                  'email_content': '<p>Trying to figure this out</p>'}

email_contact = '2015samtaylor@gmail.com'
EMAIL_ADDRESS_FROM = '2015samtaylor@gmail.com'
from config import imap_password_sam
EMAIL_PASS = imap_password_sam

SMTP_CONN = SendMail.get_smtp_connection(EMAIL_ADDRESS_FROM, EMAIL_PASS)

import pandas as pd
df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\django_testing\emailscraper_django\emailscraper_app\modules\Sending_Emails\test_emails.csv')

# blast(email_config, df, test=True)

SendMail.send(email_config, email_contact, SMTP_CONN)

# Test Requests

In [ ]:
import os
import django
from django.http import HttpRequest

# Setting up Django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'emailscraper_proj.settings')
django.setup()

# Import the views or functions you want to test
from emailscraper_app.views.uploading_file_views import serve_gcs_file, read_csv_from_gcs

# Create a mock HttpRequest object
request = HttpRequest()

# Manually populate the POST data
request.method = 'POST'
request.POST['selected_file_url'] = 'uploads/2024/8/2/email_test.csv'

# Now you can retrieve it
selected_file_url = request.POST.get('selected_file_url')

selected_file_url


In [ ]:
selected_file_url

In [ ]:
<option value="35" data-url="/serve-file/uploads/2024/8/2/email_test.csv/">uploads/2024/8/2/email_test.csv</option>

In [ ]:
import pandas as pd

# Define the data as a list of dictionaries
data = [
    {'email_test': '2015samtaylor@gmail.com', 'first_name': 'John', 'last_name': 'Doe', 'age': 28, 'city': 'New York'},
    {'email_test': '2015samtaylor@gmail.com', 'first_name': 'Jane', 'last_name': 'Smith', 'age': 34, 'city': 'Los Angeles'},
    {'email_test': '2015samtaylor@gmail.com', 'first_name': 'Alice', 'last_name': 'Johnson', 'age': 29, 'city': 'Chicago'},
    {'email_test': 'sammytaylor2006@yahoo.com', 'first_name': 'Bob', 'last_name': 'Brown', 'age': 40, 'city': 'Houston'},
    {'email_test': 'sammytaylor2006@yahoo.com', 'first_name': 'Charlie', 'last_name': 'Williams', 'age': 36, 'city': 'Phoenix'},
    {'email_test': 'sammytaylor2006@yahoo.com', 'first_name': 'David', 'last_name': 'Jones', 'age': 31, 'city': 'Philadelphia'},
    {'email_test': 'alysiakimball532@gmail.com', 'first_name': 'Alysia', 'last_name': 'Kimball', 'age': 25, 'city': 'San Diego'}
]

# Create the DataFrame
df = pd.DataFrame(data)


email_config = {'EMAIL_ADDRESS_FROM': '2015samtaylor@gmail.com', 
'EMAIL_PASS': 'feqdwowrmaqthjkx', 
'server': 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com', 
'database': 'emailcampaign', 
'table_name': 'email_history', 
'email_subject_line': 'Local Supplier for Baseball Apparel',
 'email_campaign_name': 'Local Supplier Baseball',
  'contact_column': 'email_test'}


In [ ]:

import os
import numpy as np

def get_next_50(df, email_config):


    email_contact = email_config['contact_column']

    try:    
        email_history = pd.read_csv(os.getcwd() + '\\output.csv')

        #This always needs to be 'contact_email' to refer to output.csv
        last_email_sent = email_history['contact_email'].iloc[-1]
            
        # Find the index in df where the 'email' column matches the last_email_sent
        index_to_start = df[df[email_contact] == last_email_sent].index.max() + 1

        if np.isnan(index_to_start):    #So if index_to_start is nan. Then let df be itself. 
            print('Could not match last email sent to master frame, proceeding with original frame')
            df_remaining = df
        else:
            print("Matched last email sent to master frame, proceding from last email sent")
            # # Process df starting from the index_to_start
            df_remaining = df.loc[index_to_start: index_to_start + 50]

    except FileNotFoundError:
        print('Output is not created, first run of 50 emails preparing to be sent')
        logging.info('Output is not created, first run of 50 emails preparing to be sent')

        #Process df starting from the index_to_start
        df_remaining = df.iloc[0:50]

    except TypeError:
        
        #email where final send was left off can not be matched up in the original df
        print('Email where final send was left off can not be matched up in the original df. Frame is empty')
        logging.info('Email where final send was left off can not be matched up in the original df. Frame is empty')
        #create empty frame to return
        df_remaining = pd.DataFrame()

    return(df_remaining)


get_next_50(df, email_config)

#alysia is the last email in both of my frames